### Regras


In [64]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [65]:
def analise_estoque(filial):
    df_saldo_estoque = pd.read_excel(f'planilhas/estoque/saldo_estoque_{filial}.xlsx', header=11, usecols=('B,C,F,G,H,I,J,O,P,Q,S,U,X'))[0:-3]
    df_saldo_estoque = df_saldo_estoque.set_axis(['ean','produto','laboratorio','grupo','curva','estoque_minimo','demanda','estoque','preco_custo','preco_venda','lucro','preco_custo_total','preco_venda_total'], axis=1)
    df_saldo_estoque['grupo'] = df_saldo_estoque['grupo'].astype(int)
    df_saldo_estoque['preco_custo_total'] = df_saldo_estoque['preco_custo_total'].str.replace('.', '').str.replace(',', '.')
    df_saldo_estoque['preco_custo'] = df_saldo_estoque['preco_custo'].str.replace('.', '').str.replace(',', '.')
    df_saldo_estoque['estoque_minimo'] = pd.to_numeric(df_saldo_estoque['estoque_minimo'], errors='coerce')
    df_saldo_estoque['preco_custo_total'] = pd.to_numeric(df_saldo_estoque['preco_custo_total'], errors='coerce')
    df_saldo_estoque['preco_custo'] = pd.to_numeric(df_saldo_estoque['preco_custo'], errors='coerce')
    df_saldo_estoque['ean'] = df_saldo_estoque['ean'].fillna(0).astype(int)

    df_saldo_estoque_filtrado = df_saldo_estoque.query('estoque_minimo > 0')
    valor_em_estoque = df_saldo_estoque['preco_custo_total'].sum()
    df_faltas = df_saldo_estoque_filtrado.query('estoque == 0')
    df_faltas['valor_faltas'] = df_faltas['estoque_minimo']*df_faltas['preco_custo']
    valor_faltas = df_faltas['valor_faltas'].sum()
    
    return valor_em_estoque, valor_faltas, df_saldo_estoque

def analise_estoque_grupo(df_saldo_estoque_grupo, grupo):
    df_saldo_estoque_grupo = df_saldo_estoque_grupo.query(f'grupo == {grupo}')
    df_saldo_estoque_grupo_filtrado = df_saldo_estoque_grupo.query('estoque_minimo > 0')
    valor_em_estoque = df_saldo_estoque_grupo['preco_custo_total'].sum()
    df_faltas = df_saldo_estoque_grupo_filtrado
    df_faltas['valor_faltas'] = df_faltas['estoque_minimo']*df_faltas['preco_custo']
    df_faltas['ean'] = df_faltas['ean'].astype(str)
    valor_faltas = df_faltas['valor_faltas'].sum()
    
    df_excesso = df_saldo_estoque_grupo.query('estoque > 0')
    df_excesso['excesso'] = np.where(df_excesso['estoque']>df_excesso['demanda'],df_excesso['estoque']-df_excesso['demanda'],0)
    df_excesso = df_excesso.query('excesso > 0')
    df_excesso['ean'] = df_excesso['ean'].astype(str)
    return valor_em_estoque, valor_faltas, df_faltas, df_excesso

##### somar demandas de Centrinho, Matriz e Canasvieiras

In [66]:
valor_em_estoque_matriz, valor_faltas_matriz, df_estoque_matriz = analise_estoque('001')

In [67]:
valor_em_estoque_centrinho, valor_faltas_centrinho, df_estoque_centrinho = analise_estoque('004')

In [68]:
valor_em_estoque_canasvieiras, valor_faltas_canasvieiras, df_estoque_canasvieiras = analise_estoque('009')

In [69]:
#filtrando perfumaria
df_estoque_matriz = df_estoque_matriz.query('grupo == 8000')
df_estoque_centrinho = df_estoque_centrinho.query('grupo == 8000')
df_estoque_canasvieiras = df_estoque_canasvieiras.query('grupo == 8000')

In [70]:
df_estoque_matriz = df_estoque_matriz[['ean','produto','estoque_minimo','demanda']]
df_estoque_matriz

,ean,produto,estoque_minimo,demanda
19,-2147483648,(B) = FENZZA FZ32008 BLUSH FAC,0,0.0
20,-2147483648,(B) = HB 8219 BATOM LIQUIDO ME,0,0.0
21,-2147483648,(B) = HB 8219 BATOM LIQUIDO ME,0,0.0
22,-2147483648,(L) SOS CACHOS ABACATE TONICO,0,0.0
23,-2147483648,(SELE) SLIME SABONETE LIQUIDO,0,0.0
...,...,...,...,...
22512,-2147483648,XO INSETO REPEL SPRAY 200ML,1,2.0
22518,-2147483648,YABAE MASCARA CAP CASTANHA,0,0.0
22519,-2147483648,YABAE MASCARA CAP JABORANDI 25,0,0.0
22520,-2147483648,YABAE MASCARA CAPILAR GUAXIMA,0,0.0


In [71]:
df_estoque_centrinho = df_estoque_centrinho[['ean','produto','estoque_minimo','demanda']]
df_estoque_centrinho

,ean,produto,estoque_minimo,demanda
19,-2147483648,(B) = FENZZA FZ32008 BLUSH FAC,0,0.0
20,-2147483648,(B) = HB 8219 BATOM LIQUIDO ME,0,0.0
21,-2147483648,(B) = HB 8219 BATOM LIQUIDO ME,0,0.0
22,-2147483648,(L) SOS CACHOS ABACATE TONICO,0,0.0
23,-2147483648,(SELE) SLIME SABONETE LIQUIDO,0,0.0
...,...,...,...,...
22512,-2147483648,XO INSETO REPEL SPRAY 200ML,2,3.0
22518,-2147483648,YABAE MASCARA CAP CASTANHA,0,0.0
22519,-2147483648,YABAE MASCARA CAP JABORANDI 25,0,0.0
22520,-2147483648,YABAE MASCARA CAPILAR GUAXIMA,0,0.0


In [72]:
df_estoque_canasvieiras = df_estoque_canasvieiras[['ean','produto','estoque_minimo','demanda']]
df_estoque_canasvieiras

,ean,produto,estoque_minimo,demanda
19,-2147483648,(B) = FENZZA FZ32008 BLUSH FAC,0,0.0
20,-2147483648,(B) = HB 8219 BATOM LIQUIDO ME,0,0.0
21,-2147483648,(B) = HB 8219 BATOM LIQUIDO ME,0,0.0
22,-2147483648,(L) SOS CACHOS ABACATE TONICO,0,0.0
23,-2147483648,(SELE) SLIME SABONETE LIQUIDO,0,0.0
...,...,...,...,...
22512,-2147483648,XO INSETO REPEL SPRAY 200ML,3,4.0
22518,-2147483648,YABAE MASCARA CAP CASTANHA,0,0.0
22519,-2147483648,YABAE MASCARA CAP JABORANDI 25,0,0.0
22520,-2147483648,YABAE MASCARA CAPILAR GUAXIMA,0,0.0


In [74]:
df_geral = df_estoque_matriz.merge(df_estoque_centrinho, on='ean', how='outer',)
df_geral

,ean,produto_x,estoque_minimo,demanda
17,-2147483648,(B) = FENZZA FZ32008 BLUSH FAC,1,1.0
22,-2147483648,(B) = FENZZA FZ32008 BLUSH FAC,1,1.0
23,-2147483648,(B) = FENZZA FZ32008 BLUSH FAC,1,1.0
24,-2147483648,(B) = FENZZA FZ32008 BLUSH FAC,2,2.0
25,-2147483648,(B) = FENZZA FZ32008 BLUSH FAC,1,1.0
...,...,...,...,...
140045271,-2147483648,YUDEVAN PORTA,1,1.0
140045274,-2147483648,YUDEVAN PORTA,1,1.0
140045281,-2147483648,YUDEVAN PORTA,1,1.0
140045284,-2147483648,YUDEVAN PORTA,2,2.0


In [ ]:
df_geral['estoque_minimo'] = df_geral['estoque_minimo_x'] + df_geral['estoque_minimo_y']
df_geral['demanda'] = df_geral['demanda_x'] + df_geral['demanda_y']
df_geral = df_geral[['ean','produto_x','estoque_minimo','demanda']]
df_geral

,ean,produto_x,estoque_minimo,demanda
0,-2147483648,(B) = FENZZA FZ32008 BLUSH FAC,0,0.0
1,-2147483648,(B) = FENZZA FZ32008 BLUSH FAC,0,0.0
2,-2147483648,(B) = FENZZA FZ32008 BLUSH FAC,0,0.0
3,-2147483648,(B) = FENZZA FZ32008 BLUSH FAC,0,0.0
4,-2147483648,(B) = FENZZA FZ32008 BLUSH FAC,0,0.0
...,...,...,...,...
140045285,-2147483648,YUDEVAN PORTA,2,3.0
140045286,-2147483648,YUDEVAN PORTA,0,0.0
140045287,-2147483648,YUDEVAN PORTA,0,0.0
140045288,-2147483648,YUDEVAN PORTA,0,0.0


In [ ]:
df_estoque_canasvieiras = df_estoque_canasvieiras.query('estoque_minimo > 0')

In [ ]:
df_geral = df_geral.query('estoque_minimo > 0')

In [ ]:
df_geral

,ean,produto_x,estoque_minimo,demanda
17,-2147483648,(B) = FENZZA FZ32008 BLUSH FAC,1,1.0
22,-2147483648,(B) = FENZZA FZ32008 BLUSH FAC,1,1.0
23,-2147483648,(B) = FENZZA FZ32008 BLUSH FAC,1,1.0
24,-2147483648,(B) = FENZZA FZ32008 BLUSH FAC,2,2.0
25,-2147483648,(B) = FENZZA FZ32008 BLUSH FAC,1,1.0
...,...,...,...,...
140045271,-2147483648,YUDEVAN PORTA,1,1.0
140045274,-2147483648,YUDEVAN PORTA,1,1.0
140045281,-2147483648,YUDEVAN PORTA,1,1.0
140045284,-2147483648,YUDEVAN PORTA,2,2.0


In [ ]:
df_geral = df_geral.merge(df_estoque_canasvieiras, on='ean', how='left')
df_geral

MemoryError: Unable to allocate 1.98 TiB for an array with shape (272606188366,) and data type int64